Importación de librerías base para el problema 

In [74]:
import pandas as pd
import datetime as dt
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import os

os.getcwd()


'c:\\Users\\andresfuentes\\LATAM'

Lectura de datos principal y creacion de copia auxiliar de de la data original, también se busco información extra sobre países de los destinos de vuelos y los días feriados de ese año para el origen de los vuelos

In [75]:
datos = pd.read_csv("dataset_SCL.csv", sep=",")
datos_original = datos.copy()
paises =  pd.read_excel("Prefijo Paises.xlsx", sheet_name= 'Hoja2')
feriados =  pd.read_excel("Feriados.xlsx")
datos.columns


C:\Users\andresfuentes\AppData\Local\Temp\ipykernel_2836\200450372.py:1: DtypeWarning:

Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.



Index(['Fecha-I', 'Vlo-I', 'Ori-I', 'Des-I', 'Emp-I', 'Fecha-O', 'Vlo-O',
       'Ori-O', 'Des-O', 'Emp-O', 'DIA', 'MES', 'AÑO', 'DIANOM', 'TIPOVUELO',
       'OPERA', 'SIGLAORI', 'SIGLADES'],
      dtype='object')

verificación de los tipos de datos de este dataframe

In [76]:
datos.dtypes

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

transformanción de Fechas a formato tiempo

In [77]:
datos["Fecha-I"] = pd.to_datetime(datos["Fecha-I"])
datos["Fecha-O"] = pd.to_datetime(datos["Fecha-O"])
datos.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

creación de variables dif_min, hora_dia, hora, minutos

In [78]:
datos['dif_min'] = (datos["Fecha-O"]- datos["Fecha-I"])
datos['hora_dia'] = datos['Fecha-I'].dt.time
datos['hora'] = datos['Fecha-I'].dt.hour
datos['minutos'] = datos['Fecha-I'].dt.minute

datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,-1 days +23:58:00,23:30:00,23,30


cambia la diferencia de las fechas de programación y salida de los vuelos a minutos.

In [79]:
datos["dif_min"] =  datos["dif_min"]/np.timedelta64(1,'m')

Homologación de variable Mes a nombre

In [80]:
conditions = [
    (datos['MES'] == 1),
    (datos['MES'] == 2),
    (datos['MES'] == 3),
    (datos['MES'] == 4),
    (datos['MES'] == 5),
    (datos['MES'] == 6),
    (datos['MES'] == 7),
    (datos['MES'] == 8),
    (datos['MES'] == 9),
    (datos['MES'] == 10),
    (datos['MES'] == 11),
    (datos['MES'] == 12)]
values = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre']
datos['Mes_Nombre'] = np.select(conditions, values)
datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos,Mes_Nombre
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Domingo,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Lunes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Martes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Miercoles,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Jueves,I,American Airlines,Santiago,Miami,-2.0,23:30:00,23,30,Enero


Verificacion que el valores este correcto

In [81]:
datos[["dif_min","Fecha-O","Fecha-I"]]

,dif_min,Fecha-O,Fecha-I
0,3.0,2017-01-01 23:33:00,2017-01-01 23:30:00
1,9.0,2017-01-02 23:39:00,2017-01-02 23:30:00
2,9.0,2017-01-03 23:39:00,2017-01-03 23:30:00
3,3.0,2017-01-04 23:33:00,2017-01-04 23:30:00
4,-2.0,2017-01-05 23:28:00,2017-01-05 23:30:00
...,...,...,...
68201,46.0,2017-12-22 15:41:00,2017-12-22 14:55:00
68202,16.0,2017-12-25 15:11:00,2017-12-25 14:55:00
68203,40.0,2017-12-27 15:35:00,2017-12-27 14:55:00
68204,13.0,2017-12-29 15:08:00,2017-12-29 14:55:00


Creacion de variable temporada_alta y atraso_15


In [82]:
datos['temporada_alta'] = np.where(  ((datos['Fecha-I']>='2017-12-15') & (datos['Fecha-I']<='2018-03-03')) 
                                   | ((datos['Fecha-I']>='2017-07-15') & (datos['Fecha-I']<='2017-07-31'))
                                   | ((datos['Fecha-I']>='2017-01-01') & (datos['Fecha-I']<='2017-03-03'))
                                   | ((datos['Fecha-I']>='2017-09-11') & (datos['Fecha-I']<='2017-09-30'))
                                   ,1,0)
datos['atraso_15'] = np.where(datos['dif_min']>15,1,0)




creacion de variable periodo_dia

In [83]:
conditions = [
    (datos['hora'] >= 5) & (datos['hora'] < 12),
    (datos['hora'] >= 12) & (datos['hora'] < 19),
    (datos['hora'] >= 19) | ((datos['hora'] >= 0) & (datos['hora'] < 5))]
values = ['mañana', 'tarde', 'noche']
datos['periodo_dia'] = np.select(conditions, values)


exportacion de archivo csv con las variables creadas desde el dataframe

In [84]:
synthetic_features = datos[['temporada_alta','dif_min','atraso_15','periodo_dia']]

synthetic_features.to_csv("synthetic_features.csv", sep="|"
                          , header=True, index=False)

Calculo de Proporcion de atrasos de la base total

In [85]:
Prop_atrasos = datos.groupby(['atraso_15']
                         # 'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False,as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))
Prop_atrasos['Proporción de atrasos'] = (Prop_atrasos['Cantidad_Vuelos']/68203)*100


def f(row):
    if row == 0:
        val = 'No Atraso'
    else:
        val = 'Atraso'
    return val
Prop_atrasos['atraso_15'] = Prop_atrasos['atraso_15'].apply(f)
Prop_atrasos


Prop_atrasos

,atraso_15,Cantidad_Vuelos,Proporción de atrasos
0,No Atraso,55592,81.509611
1,Atraso,12614,18.494788


Grafico Proporción de atrasos en la data completa


In [13]:

fig = px.bar(Prop_atrasos
             , y ='atraso_15'
             , x ='Proporción de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Vuelos'
             , orientation= 'h'
             , title = 'Pproporción de atrasos'
             )
fig.show()

### Se observa que el 18.49% de los vuelos generales presentan atrasos

### Calculo de proporción de atrasos por Destino, se toma en cuenta para esta proporcion destinos mayores o iguales a 10 vuelos para la variable Destino de Operacion Des-O  

In [91]:
Destino_pre = datos.groupby(['Des-O']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False, as_index= False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))


Destino_menores = Destino_pre[Destino_pre['Cantidad_Vuelos']<=10]

flex = Destino_menores['Des-O'].to_list() #agrega lista de destinos con menos de 10 vuelos
datos['Destino'] = np.where(datos['Des-O'].isin(flex), 'Otro',datos['Des-O']) #Creación de variable Destino donde se agrupan los DES-O con menos de 10 vuelos en categoria Otros


### Calculo de proporción de atrasos por Destino, se muestran el top 10 de Destinos con mayor proporción de atrasos

In [94]:
Destino = datos.groupby(['Destino']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        ,as_index=False, dropna=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))
                        
Destino['Proporción de atrasos'] = Destino['Cantidad_Atrasos']/Destino['Cantidad_Vuelos']*100 #Calculo proporción de atrasos
                       
Destino[Destino['Cantidad_Vuelos']>10].sort_values('Proporción de atrasos',ascending=False).head(10)
Destino_mayores = Destino[Destino['Cantidad_Vuelos']>10].sort_values('Proporción de atrasos',ascending=False)
Destino_mayores.sort_values('Proporción de atrasos',ascending=False).head(10)


,Destino,Cantidad_Vuelos,Cantidad_Atrasos,Proporción de atrasos
53,YSSY,194,113,58.247423
52,YMML,39,21,53.846154
17,Otro,25,10,40.000000
18,SAAR,136,54,39.705882
16,NZAA,361,135,37.396122
12,MDPC,54,18,33.333333
0,CYYZ,283,89,31.448763
20,SACO,1023,308,30.107527
46,SGAS,232,66,28.448276
27,SBGL,1435,405,28.222997


### grafíco de la proporcion de atrasos por Destino

In [95]:
fig = px.bar(Destino_mayores.head(10).sort_values('Proporción de atrasos',ascending=True)
             , y ='Destino'
             , x ='Proporción de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Atrasos'
             , orientation= 'h'
             , title = 'Ranking (Top 10) de la proporción de atrasos por destino segun cantidad de atrasos'
             )
fig.show()

### Se observa que de los Destinos de operación posibles el que presneta mayor proporcion de atrasos es el de YSSY  y YMML ambos en Australia, ¿es posible saber si Australia es el pais de destino con mayor proporción de atrasos?

 Para ellos usaremos los datos extraidos de  codificacion  Organización de Aviación Civil Internacional "OACI", La informacipon extraida de los paises tienen un prefijo para estos no todos con el mismo largo de caracteres, se hace una diferencia para poder juntar porteriomente.

In [103]:
paises['Des-O_2'] = paises['Código prefijo']
paises['Largo'] = paises['Des-O_2'].str.len()

paises_1 = paises[paises['Largo'] == 1]
paises_2 = paises[paises['Largo'] == 2]

def subs(x):
    return x[:2]
datos['Des-O_2'] = datos['Des-O'].apply(subs)
datos['Des-O_2']
paises_2 = paises_2.drop(columns=['Largo','Código prefijo'], axis=1)
paises_1 = paises_1.drop(columns=['Largo','Código prefijo'], axis=1)



#### Se crea  la variable País en la data completa

In [104]:

datos_test = datos.merge(paises_2, how='left', left_on='Des-O_2', right_on = 'Des-O_2')
datos_1 = datos_test[datos_test['País'].isnull()]
datos_2 = datos_test[datos_test['País'].notnull()]

datos_1 =  datos_1.drop(columns=[ 'País', ], axis=1)

def subs(x):
    return x[:1]
datos_1['Des-O_2'] = datos_1['Des-O'].apply(subs)
datos_1['Des-O_2']

datos_1 = datos_1.merge(paises_1, how='left', left_on='Des-O_2', right_on = 'Des-O_2')
datos_test = pd.concat([datos_1, datos_2], axis = 0)

datos_test =  datos_test.drop(columns=['LARGO_x','LARGO_y', 'LARGO'], axis=1)

datos_test =  datos_test.drop(columns=[ 'Des-O_2', ], axis=1)
datos_test


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,dif_min,hora_dia,hora,minutos,Mes_Nombre,temporada_alta,atraso_15,periodo_dia,Destino,País
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,3.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,9.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,9.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,3.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,-2.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,46.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,16.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,40.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,13.0,14:55:00,14,55,Diciembre,1,0,tarde,SPJC,Perú


#### asignare datos_test a la data con los paises asignados para mantener la data original en caso de algun problema poder volver a ella.

In [115]:
Pais = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'País'
                        ], dropna=False, as_index=False).agg(
    Atraso_count = ('atraso_15','count'),
    Atraso_sum = ('atraso_15','sum')
                                            )

Pais['Proporción de atrasos'] = (Pais['Atraso_sum']/Pais['Atraso_count'])*100

Pais = Pais.sort_values('Proporción de atrasos',ascending=False)
Pais



,País,Atraso_count,Atraso_sum,Proporción de atrasos
1,Australia,233,134,57.510730
13,Nueva Zelanda,361,135,37.396122
18,República Dominicana,54,18,33.333333
4,Canadá,283,89,31.448763
15,Paraguay,232,66,28.448276
0,Argentina,9237,2452,26.545415
6,Colombia,1604,415,25.872818
3,"Brasil (también SD, SN, SS y SW)",5171,1316,25.449623
16,Perú,5271,1214,23.031683
19,Uruguay,1072,221,20.615672


#### Calculamos la proporcion de atraso por Pais de destino y podemos obersvar que efevtivamente Australia es el pais que presenta la mayor porocion de atrasos en los viajes. 

In [116]:
fig = px.bar(Pais.sort_values('Proporción de atrasos',ascending=True), y='País', x='Proporción de atrasos', color='Atraso_sum')
fig.show()

### Gráfico de proporcion de atrasos por país de Destino coloreado segun la suma de los atrasos. esto nos dice que la la proporcion de atrasos mayor la presenta Australia como país, pero la mayor cantidad de atrasos la tiene Chile

#### Analísis de proporción de atrasos segun tipo de vuelo, Internacional o Nacional.

In [112]:
Tipo_Vuelo = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'TIPOVUELO'
                        ], dropna=False, as_index=False).agg(
    Atraso_count = ('atraso_15','count'),
    Atraso_sum = ('atraso_15','sum')
                                            )

Tipo_Vuelo['Proporción de atrasos'] = (Tipo_Vuelo['Atraso_sum']/Tipo_Vuelo['Atraso_count'])*100




def f(row):
    if row == 'N':
        val = 'Nacional'
    else:
        val = 'Internacional'
    return val
    
Tipo_Vuelo['TIPOVUELO'] = Tipo_Vuelo['TIPOVUELO'].apply(f)
Tipo_Vuelo



,TIPOVUELO,Atraso_count,Atraso_sum,Proporción de atrasos
0,Internacional,31240,7048,22.560819
1,Nacional,36966,5566,15.057079


In [122]:
fig = px.bar(Tipo_Vuelo.sort_values('Proporción de atrasos',ascending=True), y = 'TIPOVUELO', x = 'Proporción de atrasos', color = 'Atraso_sum',text_auto=True)

fig.update_traces(textfont_size=8
                  , textangle=45
                  , textposition="inside"
                  ,marker_line_width=1
                  ,marker_line_color = 'black')

fig.show()

### Se observa que en los Vuelos Nacionales existe un 22.56% de proporción de atrasos vs un 15.05% de proporcion de atrasos en los vuelos internacionales.